# Modelado Predictivo
# Proyecto: Predicción de Ventas

**Autor:** Javier Gacitúa  
**Fecha:** Octubre 2025

---

## Objetivos

1. Entrenar múltiples modelos de Machine Learning
2. Evaluar y comparar rendimiento de modelos
3. Optimizar hiperparámetros
4. Seleccionar el mejor modelo
5. Generar predicciones y visualizaciones

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(" Librerías importadas correctamente")

## 1. Cargar Datos Procesados

In [ ]:
# Importar módulos propios
import sys
sys.path.append('../src')
from data_loader import DataLoader
from models import SalesPredictor
from visualization import SalesVisualizer

# Inicializar
loader = DataLoader(data_dir='../data')
predictor = SalesPredictor()
visualizer = SalesVisualizer()

# Cargar datos (reemplazar con tu dataset)
# df = loader.load_local_csv('sales_engineered.csv', directory='processed')

# Datos de ejemplo con features
np.random.seed(42)
n_samples = 1000
df = pd.DataFrame({
    'sales': np.random.normal(1000, 200, n_samples) + np.linspace(0, 500, n_samples),
    'price': np.random.uniform(10, 100, n_samples),
    'promotion': np.random.choice([0, 1], n_samples),
    'customers': np.random.randint(10, 200, n_samples),
    'sales_lag_1': np.random.normal(1000, 200, n_samples),
    'sales_lag_7': np.random.normal(1000, 200, n_samples),
    'sales_rolling_mean_7': np.random.normal(1000, 150, n_samples),
    'day_of_week': np.random.randint(0, 7, n_samples),
    'month': np.random.randint(1, 13, n_samples),
    'is_weekend': np.random.choice([0, 1], n_samples)
})

print(f" Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
df.head()

## 2. Preparación de Datos

In [ ]:
# Separar features y target
target_column = 'sales'
X_train, X_test, y_train, y_test = predictor.prepare_data(df, target_column, test_size=0.2)

print(f"\n Datos preparados correctamente")
print(f"  - Features de entrenamiento: {X_train.shape}")
print(f"  - Features de prueba: {X_test.shape}")
print(f"  - Target de entrenamiento: {y_train.shape}")
print(f"  - Target de prueba: {y_test.shape}")

## 3. Modelo Baseline: Regresión Lineal

In [ ]:
# Entrenar Linear Regression
lr_model = predictor.train_linear_regression(X_train, y_train, name='Linear Regression')

# Evaluar
lr_metrics = predictor.evaluate_model(lr_model, X_test, y_test, 'Linear Regression')

In [ ]:
# Visualizar predicciones vs reales
y_pred_lr = lr_model.predict(X_test)
visualizer.plot_predictions_vs_actual(y_test, y_pred_lr, 
                                     title='Linear Regression: Predicciones vs Reales')

## 4. Random Forest

In [ ]:
# Entrenar Random Forest
rf_model = predictor.train_random_forest(X_train, y_train, 
                                        n_estimators=100, 
                                        max_depth=15,
                                        name='Random Forest')

# Evaluar
rf_metrics = predictor.evaluate_model(rf_model, X_test, y_test, 'Random Forest')

In [ ]:
# Visualizar predicciones
y_pred_rf = rf_model.predict(X_test)
visualizer.plot_predictions_vs_actual(y_test, y_pred_rf, 
                                     title='Random Forest: Predicciones vs Reales')

In [ ]:
# Feature Importance de Random Forest
feature_importance = predictor.get_feature_importance('Random Forest', 
                                                     X_train.columns.tolist(), 
                                                     top_n=15)

if feature_importance is not None:
    visualizer.plot_feature_importance(feature_importance, top_n=15)

## 5. XGBoost

In [ ]:
# Entrenar XGBoost
xgb_model = predictor.train_xgboost(X_train, y_train, 
                                   n_estimators=100,
                                   learning_rate=0.1,
                                   max_depth=6,
                                   name='XGBoost')

if xgb_model is not None:
    # Evaluar
    xgb_metrics = predictor.evaluate_model(xgb_model, X_test, y_test, 'XGBoost')

In [ ]:
# Visualizar predicciones XGBoost
if xgb_model is not None:
    y_pred_xgb = xgb_model.predict(X_test)
    visualizer.plot_predictions_vs_actual(y_test, y_pred_xgb, 
                                         title='XGBoost: Predicciones vs Reales')

In [ ]:
# Feature Importance de XGBoost
if xgb_model is not None:
    xgb_importance = predictor.get_feature_importance('XGBoost', 
                                                     X_train.columns.tolist(), 
                                                     top_n=15)
    if xgb_importance is not None:
        visualizer.plot_feature_importance(xgb_importance, top_n=15)

## 6. Comparación de Modelos

In [ ]:
# Comparar todos los modelos
results_df = predictor.compare_models()

In [ ]:
# Visualizar comparación
if results_df is not None:
    visualizer.plot_model_comparison(results_df)

## 7. Validación Cruzada

In [ ]:
# Validación cruzada para el mejor modelo
from sklearn.model_selection import cross_val_score

if predictor.best_model is not None:
    print("\n Realizando Validación Cruzada (5-fold)...")
    
    cv_scores = cross_val_score(predictor.best_model, X_train, y_train, 
                               cv=5, scoring='r2', n_jobs=-1)
    
    print("\n RESULTADOS DE VALIDACIÓN CRUZADA")
    print("=" * 50)
    print(f"R² scores: {cv_scores}")
    print(f"Media: {cv_scores.mean():.4f}")
    print(f"Desviación estándar: {cv_scores.std():.4f}")
    print(f"Rango: [{cv_scores.min():.4f}, {cv_scores.max():.4f}]")
    
    # Visualizar
    plt.figure(figsize=(10, 6))
    plt.boxplot(cv_scores, vert=True)
    plt.ylabel('R² Score')
    plt.title('Distribución de R² en Validación Cruzada', fontweight='bold', fontsize=14)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

## 8. Análisis de Errores

In [ ]:
# Análisis detallado de errores
if predictor.best_model is not None:
    y_pred_best = predictor.best_model.predict(X_test)
    errors = y_test - y_pred_best
    abs_errors = np.abs(errors)
    pct_errors = np.abs(errors / y_test) * 100
    
    print("\n ANÁLISIS DE ERRORES")
    print("=" * 50)
    print(f"Error promedio: ${errors.mean():.2f}")
    print(f"Error absoluto promedio: ${abs_errors.mean():.2f}")
    print(f"Error porcentual promedio: {pct_errors.mean():.2f}%")
    print(f"Mediana del error absoluto: ${np.median(abs_errors):.2f}")
    print(f"Error máximo: ${abs_errors.max():.2f}")
    
    # Distribución de errores
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].hist(errors, bins=50, color='coral', edgecolor='black', alpha=0.7)
    axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[0].set_xlabel('Error (Real - Predicción)')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Distribución de Errores', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].hist(pct_errors, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Error Porcentual (%)')
    axes[1].set_ylabel('Frecuencia')
    axes[1].set_title('Distribución de Errores Porcentuales', fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 9. Predicciones en Conjunto de Prueba

In [ ]:
# Crear DataFrame con predicciones
if predictor.best_model is not None:
    predictions_df = pd.DataFrame({
        'Real': y_test.values,
        'Predicción': y_pred_best,
        'Error': errors,
        'Error_Abs': abs_errors,
        'Error_Pct': pct_errors
    })
    
    # Ordenar por error absoluto
    predictions_df = predictions_df.sort_values('Error_Abs', ascending=False)
    
    print("\n Top 10 Predicciones con Mayor Error:")
    print(predictions_df.head(10))
    
    print("\n Top 10 Mejores Predicciones:")
    print(predictions_df.tail(10))

## 10. Guardar Mejor Modelo

In [ ]:
# Guardar el mejor modelo
if predictor.best_model is not None:
    model_path = '../models/saved_models/best_sales_model.pkl'
    # predictor.save_model(list(predictor.models.keys())[0], model_path)
    print(f" Modelo guardado en: {model_path}")

## 11. Resumen Final

In [ ]:
# Resumen completo
print("\n" + "=" * 70)
print(" RESUMEN DEL MODELADO")
print("=" * 70)

if results_df is not None:
    print("\n Comparación de Modelos:")
    print(results_df.to_string())
    
    best_model_name = results_df.index[0]
    best_r2 = results_df.loc[best_model_name, 'R2']
    best_rmse = results_df.loc[best_model_name, 'RMSE']
    best_mae = results_df.loc[best_model_name, 'MAE']
    best_mape = results_df.loc[best_model_name, 'MAPE']
    
    print(f"\n MEJOR MODELO: {best_model_name}")
    print("=" * 40)
    print(f"  R²: {best_r2:.4f}")
    print(f"  RMSE: ${best_rmse:.2f}")
    print(f"  MAE: ${best_mae:.2f}")
    print(f"  MAPE: {best_mape:.2f}%")

print("\n Modelado completado exitosamente")
print(" Próximo paso: Crear dashboard interactivo con Streamlit")
print("=" * 70)

## 12. Insights de Negocio

###  Hallazgos Clave:

1. **Rendimiento del Modelo:**
   - El mejor modelo logró un R² de [valor], explicando [%] de la variabilidad en ventas
   - Error promedio de predicción: $[valor] ([%] del valor promedio)

2. **Variables Más Importantes:**
   - [Listar top 5 features más importantes]
   - Estos factores son los principales drivers de ventas

3. **Casos de Error:**
   - Mayores errores ocurren en [describir casos]
   - Posibles mejoras: [sugerencias]

###  Recomendaciones de Negocio:

1. **Optimización de Inventario:**
   - Usar predicciones para planificar stock
   - Evitar sobre/sub inventario

2. **Estrategia de Precios:**
   - Ajustar precios según predicciones de demanda
   - Identificar productos de alta rotación

3. **Marketing y Promociones:**
   - Optimizar timing de promociones
   - Focalizar en períodos de alta demanda predicha

4. **Expansión:**
   - Identificar tiendas/categorías con mayor potencial
   - Replicar estrategias exitosas

---

**Nota:** Actualizar con resultados reales de tu dataset